# Creating and Visualizing DEMs from LIDAR points

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_

In this notebook we will:
* Create high-quality DEMs from LiDAR point clouds and compute topographic parameters
* Create webmap visualizations


## 1. Startup

Import Python standard library and IPython packages we need. Start GRASS session in Nags Head project.

In [9]:
import subprocess
import sys
from pathlib import Path

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

import grass.script as gs
import grass.jupyter as gj

gj.init("./nags_head/PERMANENT");

In [10]:
!g.region -p

projection: 99 (NAD83(HARN) / North Carolina)
zone:       0
datum:      nad83harn
ellipsoid:  grs80
north:      250984
south:      249364
west:       912140
east:       914462
nsres:      1
ewres:      1
rows:       1620
cols:       2322
cells:      3761640


## 3. Creating DEMs

In [ ]:
lidar_files = sorted(Path('./Jockeys_Ridge_LiDAR/').glob('*.txt'))
lidar_files

### Mask low density areas

Interpolating a DEM from LiDAR points is only meaningful where there are adequate point coverage. So, our first step will be to mask areas that have low point densities before we interpolate. This also will make our interpolation run faster!

In [ ]:
!g.region raster="naip_2020.1" res=5

In [ ]:
gs.run_command("r.in.xyz", input=lidar_files[0], output="dem_1999_raw", method="n", fs=",")
gs.mapcalc(exp="dem_1999_mask=if(dem_1999_raw == 0, null(), 1 )")

In [ ]:
mask = gj.Map()
mask.d_rgb(red="naip_2020.1", green="naip_2020.2", blue="naip_2020.3")
mask.d_rast(map="dem_1999_mask")
mask.show()

In [ ]:
!g.region res=0.5
!r.mask raster=dem_1999_mask

### Creating DEMs with Binning

In [ ]:
# Binning
!g.region res=2

In [ ]:
gs.run_command("r.in.xyz", input=lidar_files[0], output="dem_1999", method="mean", separator="comma")

For you can also compute DEMs with a higher resolution than the point sampling distance using splines with either the [v.surf.rst](https://grass.osgeo.org/grass83/manuals/v.surf.rst.html) or [v.surf.bspline](https://grass.osgeo.org/grass83/manuals/v.surf.bspline.html) tools.

See Section 2.3.3 of [Hardin et al (2014)](https://link.springer.com/chapter/10.1007/978-1-4939-1835-5_2#Sec3) for more.

In [ ]:
mask = gj.Map()
mask.d_rast(map="dem_1999")
mask.show()